In [9]:
import pandas as pd
import numpy as np
import json
import os

import warnings
warnings.filterwarnings("ignore")

In [10]:
os.chdir(r"D:\A李杜若\留学\科研\可视化_王懿芳学姐\CFPS\NLS\1NLSY79")

In [3]:
d = {}
d['person_id']=''
d['basic_info']={"gender":"",
                 "age":0, #birth_year
                  "race":""}

d["education"] = { 
          "year":0,
          "highest_education_rank":""
        }

d["life_sequence"] =  []
ls = { "year":0,
    "marriage_status":"",
    "marriage_status_change":False,
    "childbearing_status":True,
    "child_under_18":True,
    'child_under_18_status_change':None, #0,1,2
    "residence":"",
    "migration_status":True,
    
    "isWorking":True,
    "income":0,
    "occupation":0
  }

d["spouse_info"]= {
  "spouse_id":"",
  "spouse_age":0,
  "spouse_race":"",
  "spouse_education": {
    "year",
    "highest_education_rank",
  },
  "spouse_first_job": {
    "year":0,
    "income":0,
    "occupation":0,
  },
  "spouse_life_sequence":[]
}

## input

In [11]:
basic_info = pd.read_csv("./basic_info.csv")
edu_info = pd.read_csv("./edu_info.csv")
job_info = pd.read_csv("./job_info.csv")
mar_info = pd.read_csv("./marry_info.csv")
res_info = pd.read_csv('./residence_info.csv')
childbirth_info = pd.read_csv('./childbirth_year_info.csv')

In [5]:
df_id = basic_info[["R0000100"]]

In [12]:
n = basic_info.shape[0]
# create dictionary list
d = {}
d['person_id']=''
d['basic_info']={
    "gender":"",
    "age":0, #birth_year
    "race":"",

    "education": { 
        "year":0,
        "highest_education_level": "", 
        "highest_education_name": "",
        },
    
    "first_job": {
        "year":0,
        "income":0,
        "occupation":0,
        }
    }
d["life_sequence"] =  []
ls = { "year":0,
    "marriage_status":"",
    "marriage_status_change":False,
    "childbearing_status":True,
    "child_under_18":True,
    'child_under_18_status_change':None, #0,1,2
    "residence":"",
    "migration_status":True,
    
    "isWorking":True,
    "income":0,
    "occupation":0
}

d["spouse_info"]= {
"spouse_id":"",
"spouse_age":0,
"spouse_race":"",
"spouse_education": {
    "year":0,
    "highest_education_level": "", 
    "highest_education_name": "",
},
"spouse_first_job": {
    "year":0,
    "income":0,
    "occupation":0,
},
"spouse_life_sequence":[]
}

people_dict = [{"person_id": str(i+1), 'basic_info': {}, 'life_sequence': [], 'spouse_info': {}} for i in range(n)]

## gender, birth, race

In [13]:
#1. gender, birth_year, race
gender_map = {
    1: 'male',
    2: 'female'
}
race_map = {
    1: 'hispanic',
    2: 'black',
    3: 'other'
}
for idx, person in enumerate(people_dict):
    gender = basic_info['R0214800'][idx]
    year = basic_info['R0000500'][idx]
    race = basic_info['R0214700'][idx]
    if gender < 0: 
        people_dict[idx]['basic_info']['gender'] = None
    else:
        people_dict[idx]['basic_info']['gender'] = gender_map[gender]
    if year < 0: 
        people_dict[idx]['basic_info']['birth_year'] = None
    else:
        people_dict[idx]['basic_info']['birth_year'] = int(year + 1900)
    if race < 0: 
        people_dict[idx]['basic_info']['race'] = None
    else:
        people_dict[idx]['basic_info']['race'] = race_map[race]

## education

In [14]:
#education
#'T9900000':highest_degree_level
#'T1215701': year get the highest degree 
#'T9900000-l': highest_degree_name (created)
edu_info[['T9900000-l']]=None
for i in range(n):
    if 12<=edu_info['T9900000'][i]<=15:
        edu_info['T9900000-l'][i]= "high-school"
    elif 16<=edu_info['T9900000'][i]<=20:
        edu_info['T9900000-l'][i]= "college"
    elif (1<=edu_info['T9900000'][i]<=11)|(93<=edu_info['T9900000'][i]<=95):
        edu_info['T9900000-l'][i]= "high-school-noncompletion" 
   

In [15]:
index = [0,93,94,95]+[i for i in range(1,21)]
name = [None, 'pre_kindergarten', 'kindergarten','ungraded']+['1st grade','2nd grade','3rd grade']+[str(i)+'th grade' for i in range(4,13)]+['1st year college','2nd year college','3rd year college']+[str(i)+'th year college' for i in range(4,8)]+['8th year college or more']
degree_map = dict(zip(index,name))

In [16]:
for idx, person in enumerate(people_dict):
    edu_dict = {'year': None, 'highest_education_level': None, 'highest_education_name': None}
    if edu_info['T9900000'][idx]>0:
        if edu_info['T1215701'][idx]>0:
            edu_dict['year'] = int(edu_info['T1215701'][idx])#json doesn't accept int64
        edu_dict['highest_education_name'] = degree_map[edu_info['T9900000'][idx]]
        edu_dict['highest_education_level'] = edu_info['T9900000-l'][idx]
    person['basic_info']['education'] = edu_dict
    

## first_job

In [17]:
occu_code_list = ['R0046400', 'R0263400', 'R0446400', 'R0702100', 'R0945000',
       'R1255400', 'R1650200', 'R1922800', 'R2317600', 'R2525400', 'R2924400',
       'R3127100', 'R3522800', 'R3727800', 'R4182100', 'R4587904', 'R5270600',
       'R6472600', 'R6591800', 'R7209600', 'R7898000', 'T0138400', 'T1298000',
       'T2326500', 'T3308700', 'T4282800', 'T5256900', 'T7818600']

inc_code_list = ['R0312300',
       'R0482600', 'R0782101', 'R1024001', 'R1410701', 'R1778501', 'R2141601',
       'R2350301', 'R2722501', 'R2971401', 'R3279401', 'R3559001', 'R3897101',
       'R4295101', 'R4982801', 'R5626201', 'R6364601', 'R6909701', 'R7607800',
       'R8316300', 'T0912400', 'T2076700', 'T3045300', 'T3977400', 'T4915800',
       'T5619500', 'T8115400', 'income_2018']
year_list = [1979,	1980,	1981,	1982,	1983,	1984,	1985,	1986,	1987,	1988,	1989,	1990,	1991,	1992,	1993,	1994,	1996,	1998,	2000,	2002,	2004,	2006,	2008,	2010,	2012,	2014,	2016,	2018]

for idx, person in enumerate(people_dict):
    first_job_dict = {'year': None, 'income': None, 'occupation': None}
    for y_idx, year in enumerate(year_list):
        occu = job_info[occu_code_list[y_idx]][idx]
        income = job_info[inc_code_list[y_idx]][idx]
        if occu > 0 or income > 0:
            first_job_dict['year'] = int(year)
            if occu > 0:
                first_job_dict['occupation'] = int(occu)
            if income > 0:
                first_job_dict['income'] = int(income)
            break       
    person['basic_info']['first_job'] = first_job_dict

## life_sequence

In [18]:
res_code_list = ['R0215200', 'R0393520', 'R0647000', 'R0897900', 'R1146400',
       'R1521700', 'R1892400', 'R2259500', 'R2447000', 'R2872800', 'R3076500',
       'R3403200', 'R3658600', 'R4009100', 'R4420200', 'R5083200', 'R5168500',
       'R6481300', 'R7009000', 'R7706300', 'R8498700', 'T0990500', 'T2212300',
       'T3110200', 'T4114700', 'T5026000', 'T5774100', 'T8221300']

mar_status_code_list = ['R0217501', 'R0405601', 'R0618601', 'R0898401', 'R1144901',
       'R1520101', 'R1890801', 'R2257901', 'R2445301', 'R2871000', 'R3074700',
       'R3401400', 'R3656800', 'R4007300', 'R4418400', 'R5081400', 'R5166700',
       'R6479300', 'R7007000', 'R7704300', 'R8496700', 'T0988500', 'T2210500',
       'T3108400', 'T4112900', 'T5023300', 'T5771200', 'T8219300']

occu_code_list = ['R0046400', 'R0263400', 'R0446400', 'R0702100', 'R0945000',
       'R1255400', 'R1650200', 'R1922800', 'R2317600', 'R2525400', 'R2924400',
       'R3127100', 'R3522800', 'R3727800', 'R4182100', 'R4587904', 'R5270600',
       'R6472600', 'R6591800', 'R7209600', 'R7898000', 'T0138400', 'T1298000',
       'T2326500', 'T3308700', 'T4282800', 'T5256900', 'T7818600']

inc_code_list = ['R0312300',
       'R0482600', 'R0782101', 'R1024001', 'R1410701', 'R1778501', 'R2141601',
       'R2350301', 'R2722501', 'R2971401', 'R3279401', 'R3559001', 'R3897101',
       'R4295101', 'R4982801', 'R5626201', 'R6364601', 'R6909701', 'R7607800',
       'R8316300', 'T0912400', 'T2076700', 'T3045300', 'T3977400', 'T4915800',
       'T5619500', 'T8115400', 'income_2018']

chb_code_list = ['R9900002', 'R9900802', 'R9901602', 'R9902402', 'R9903202','R9904002', 'R9904802', 'R9905602', 'R9906202', 'R9906802', 'R9907402']

year_list = [1979,	1980,	1981,	1982,	1983,	1984,	1985,	1986,	1987,	1988,	1989,	1990,	1991,	1992,	1993,	1994,	1996,	1998,	2000,	2002,	2004,	2006,	2008,	2010,	2012,	2014,	2016,	2018]

mar_map = {
    0: 'never married',
    1: 'married',
    2: 'seperated',
    3: 'divorced',
    5: 'remarried',
    6: 'widowed'
}
# 1979-1998
res_map1 = {
    0: 'not in SMSA',
    1: 'SMSA, not in central city',
    2: 'SMSE,central city not known',
    3: 'SMSA, in central city'
}
# 2000-2018 different map
res_map2 = {
    1: ' ',
    2: 'SMSA, not in central city',
    3: 'SMSE,central city not known',
    4: 'SMSA, in central city'
}
# childbirth_year_map: 从1994开始，直到下一个调查年之前，都算做前一个调查年，如1995年的childbirth算作1994年，这样它会被算入对1996年的影响
key_y = [i for i in range(1979,2019)]
value_y = [i for i in range(1979,1994)]+list(np.repeat([i for i in range(1994,2020,2)],2))+[2018]
childbirth_map = dict(zip(key_y,value_y))

In [19]:
for idx, person in enumerate(people_dict):
    person['life_sequence']=[]
# get childbirth_year data raw list & processed list
    child_list = childbirth_info.iloc[idx,1:]
    child_year_list0 = list(filter(lambda x: (x>0),child_list))
    child_year_list1 = child_year_list0.copy()
    for ch_idx, ch_year in enumerate(child_year_list0):
           if ch_year>=1979:
                  child_year_list1[ch_idx] = childbirth_map[ch_year]
    

    for y_idx, year in enumerate(year_list):
       year_dict = {'year': year, 'marriage_status': None, 'marriage_status_change': None, 'childbearing_status': None, 'child_under_18': None, 'child_under_18_status_change': None, 'residence': None, 'migration_status': None, 'isWorking': None, 'occupation': None, 'income': None 
       }
# marital status
       marital_status = mar_info[mar_status_code_list[y_idx]][idx]
       if marital_status >= 0:
              year_dict['marriage_status'] = mar_map[marital_status]
#marital_status_change
       if y_idx == 0:
              year_dict['marriage_status_change'] = -1
       if y_idx > 0 and person['life_sequence'][y_idx - 1]['marriage_status'] != None and year_dict['marriage_status'] != None:
              if person['life_sequence'][y_idx - 1]['marriage_status'] != year_dict['marriage_status']:
                     year_dict['marriage_status_change'] = True
              else: 
                     year_dict['marriage_status_change'] = False
# childbearing_status
       if len(child_year_list0) > 0:
              if year in child_year_list1:
                     year_dict['childbearing_status'] = True
              else:
                     year_dict['childbearing_status'] = False
# child_under_18_status
       if len(child_year_list0) > 0:
             before_year_list = list(filter(lambda x: (x <= year),child_year_list0)) 
             if len(before_year_list)==0:
                    year_dict['child_under_18'] = False
             elif 0<=(year-max(before_year_list))< 18:
                    year_dict['child_under_18'] = True
             else:
                    year_dict['child_under_18'] = False
#child_under_18_status_change
# 0: 状态不变；
# 1: 有小孩(<18岁)--> 无小孩；
# 2: 无小孩-->有小孩
       if y_idx == 0:
              year_dict['child_under_18_status_change'] = -1
       if y_idx > 0 and person['life_sequence'][y_idx - 1]['child_under_18'] != None and year_dict['child_under_18'] != None:
              if person['life_sequence'][y_idx - 1]['child_under_18'] == year_dict['marriage_status']:
                     year_dict['child_under_18_status_change'] = 0
              if person['life_sequence'][y_idx - 1]['child_under_18'] == True and year_dict['marriage_status'] == False:
                     year_dict['child_under_18_status_change'] = 1
              if person['life_sequence'][y_idx - 1]['child_under_18'] == False and year_dict['marriage_status'] == True:
                     year_dict['child_under_18_status_change'] = 2          
# residence
# IS R'S CURRENT RESIDENCE IN SMSA? 
       residence = res_info[res_code_list[y_idx]][idx]
       if year < 2000 and residence >= 0:
              year_dict['residence'] = res_map1[residence]
       if year >= 2000 and residence >= 0:
              year_dict['residence'] = res_map2[residence]       
# migration_status
       if y_idx == 0:
              year_dict['migration_status'] = -1
       if y_idx > 0 and person['life_sequence'][y_idx - 1]['residence'] != None and year_dict['residence'] != None:
              if person['life_sequence'][y_idx - 1]['residence'] != year_dict['residence']:
                     year_dict['migration_status'] = True
              else:
                     year_dict['migration_status'] = False
## job information
       occu = job_info[occu_code_list[y_idx]][idx]
       income = job_info[inc_code_list[y_idx]][idx]
# isworking 
       if occu > 0 or income > 0:
              year_dict['isWorking'] = True
       else:
              year_dict['isWorking'] = False
# occupation
       if occu > 0:
              year_dict['occupation'] = int(occu)
# income
       if income > 0:
              year_dict['income'] = int(income)  
              
## 逐个加入个人的life_sequence              
       person['life_sequence'].append(year_dict)        

## spouse_info

In [20]:
## Create spouse_id df
df_hh=pd.read_csv("./household_spouse_id.csv")
df_spid = pd.DataFrame(columns=['R0000100','SP_ID'])
index_rela = ['R0000151','R0000153','R0000155','R0000157']
index_spid = ['R0000150','R0000152','R0000154','R0000156']
for i in range(4):
    d = df_hh[df_hh[index_rela[i]]==1]
    dd = d[['R0000100', index_spid[i]]].rename({index_spid[i]:'SP_ID'}, axis=1)
    df_spid = df_spid.append(dd)
# create spouse_id map
spouse_id_map = dict(zip(df_spid['R0000100'], df_spid['SP_ID']))
pid_list = df_spid['R0000100']

In [21]:
for idx, person in enumerate(people_dict):
    person['spouse_info']={
        "spouse_id":None,
        "spouse_birth_year":None,
        "spouse_race":None,
        "spouse_education": None,
        "spouse_first_job": None,
        "spouse_life_sequence":[]
    }
    
    for sp_idx, pid in enumerate(pid_list):
        sp_pid = spouse_id_map[pid] 
        person['spouse_info']['spouse_id'] = int(sp_pid)
        person['spouse_info']['spouse_birth_year'] = people_dict[sp_pid]['basic_info']['birth_year']
        person['spouse_info']['spouse_race'] = people_dict[sp_pid]['basic_info']['race']
        person['spouse_info']['spouse_education'] = people_dict[sp_pid]['basic_info']['education']
        person['spouse_info']['spouse_first_job'] = people_dict[sp_pid]['basic_info']['first_job']
        person['spouse_info']['spouse_life_sequence'] = people_dict[sp_pid]['life_sequence']

## *预处理：合并occu+income处理

In [30]:
df0=pd.read_csv(r"D:\A李杜若\留学\科研\可视化_王懿芳学姐\CFPS\NLS\NLS79_v2\NLS_v2.csv")
occu = ['R0000100',
  'R0046400',
  'R0263400',
  'R0446400',
  'R0702100',
  'R0945000',
  'R1255400',
  'R1650200',
  'R1922800',
  'R2317600',
  'R2525400',
  'R2924400',
  'R3127100',
  'R3522800',
  'R3727800',
  'R4182100',
  'R4587904',
  'R5270600',
  'R6472600',
  'R6591800',
  'R7209600',
  'R7898000',
  'T0138400',
  'T1298000',
  'T2326500',
  'T3308700',
  'T4282800',
  'T5256900',
  'T7818600']

df_occu = df0[occu]

In [31]:
df_inc = pd.read_csv(r"D:\A李杜若\留学\科研\可视化_王懿芳学姐\CFPS\NLS\NLS79_v2\R_income.csv")
df_inc1 = df_inc.drop_duplicates(subset="R0000100", keep='first', inplace=False)
df_inc1 = df_inc1.drop('R0155400',axis=1)
df_inc1['income_2018']=df_inc1.iloc[:,-1]
c = pd.merge(df_occu,df_inc1, on='R0000100', how='outer').fillna(-1)
c.to_csv("./job_info.csv",index=False)

## output

In [312]:
##转化np值为原始值 e.g. int64 -> int
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, np.bool_):
            return super().encode(bool(obj))
        else:
            return super(NpEncoder, self).default(obj)

class CustomJSONizer(json.JSONEncoder):
    def default(self, obj):
        return super().encode(bool(obj)) \
            if isinstance(obj, np.bool_) \
            else super().default(obj)

In [22]:
with open("./NLSY79_clean.json", 'w') as f:
    json.dump(people_dict,f,indent = 2,sort_keys=True)

## Explore

### 婚姻情况变化

In [16]:
##婚姻状况不变 vs 变化
mar_status_change_sum = {
    'not change': 0,
    'change': 0,
    'None': 0
}
i=k=j=0
for idx, person in enumerate(people_dict):
    mar_ch_list = []
    y_n = len(person['life_sequence'])
    for y_idx, year in enumerate(person['life_sequence']):
        mar_ch = year['marriage_status_change']
        if mar_ch != None:
                mar_ch_list.append(mar_ch)
    if len(mar_ch_list)<y_n:
        k=k+1
    elif True in mar_ch_list:
        j=j+1
    else:
        i=i+1
    mar_status_change_sum['change']=i
    mar_status_change_sum['not change']=j
    mar_status_change_sum['None']=k

In [17]:
mar_status_change_sum # 77.0% not change

{'not change': 291, 'change': 110, 'None': 12285}

### 计算Event数目

In [18]:
indice = list(range(len(year_list)))
year_value = year_list
idx_year_map = dict(zip(indice,year_value))
have_1st_job_list = [] #每个人的event dict 的 list

for idx, person in enumerate(people_dict):
    total_num = 0
    event_dict = {
        'person_id': None,
        'first_job_year':None,
        'event_n': None,
# before 1st job:0
# after 1st job:1
        "marriage_status_change":{
            'n':None,
            'happen_time':None
        }, 
        "childbearing_status":{
            'n':None,
            'happen_time':None
        },
        'child_under_18_status_change':{
            'n':None,
            'happen_time':None
        },
        'migration_status':{
            'n':None,
            'happen_time':None
        },
    }
# only take person who has first_job record
    job_year = person['basic_info']['first_job']['year']
    if job_year!=None:
        event_dict['person_id'] = person['person_id']
        event_dict['first_job_year'] = job_year
    
        df = pd.DataFrame.from_dict(person['life_sequence'])
        df = df.replace({np.nan: None})
    # status_change events(marriage, child_u18, migration)
    # childbearing 没有-1
    # len(!=0)-1>=1, since first is -1
        status_event_list = ["marriage_status_change", "childbearing_status", 'child_under_18_status_change', "migration_status"]
        for e_idx, event in enumerate(status_event_list):
            event_list = df[event]
            happen_list = list(filter(None, event_list))
            if (-1 in happen_list):
                happen_list.remove(-1)
            num = len(happen_list)
            if num > 0:
                event_dict[event]['n'] = num
                total_num = total_num + num
            # get all index of True
                y_idx_list = [i for i,val in enumerate(event_list) if (val!=None and val!=0 and val!=-1)]
            # < 1st job year:'before' vs >= 1st job year:'after'
                comp_list = []
                for y_idx in y_idx_list:
                    if idx_year_map[y_idx] < job_year:
                        comp_dict = {
                            'year': int(idx_year_map[y_idx]),
                            'compare': 'before' }
                    else:
                        comp_dict = {
                            'year': int(idx_year_map[y_idx]),
                            'compare': 'after' }
                    comp_list.append(comp_dict)       
                    
                event_dict[event]['happen_time'] = comp_list

        
        event_dict['event_n'] = total_num
        have_1st_job_list.append(event_dict)           

            

In [23]:
with open("./NLSY79_event.json", 'w') as f:
    json.dump(have_1st_job_list,f, indent = 2, sort_keys=True)

In [19]:
# event number >=1
len([i for i in have_1st_job_list if i['event_n']>0])

11431

In [20]:
# event number >=2
len([i for i in have_1st_job_list if i['event_n']>1])

10393

In [21]:
# event number >=3
len([i for i in have_1st_job_list if i['event_n']>2 ])

9119

In [22]:
## occupation/income每年都有有效值
## 得到 person_id
occu_full_id_list = []
income_full_id_list = []
for idx, person in enumerate(people_dict):
    df = pd.DataFrame.from_dict(person['life_sequence']).replace({np.nan: None})
    occu_sequence = list(df['occupation'])
    if None not in occu_sequence:
        occu_full_id_list.append(person['person_id'])
    income_sequence = df['income']
    if None not in list(income_sequence):
        income_full_id_list.append(person['person_id'])
  
len(occu_full_id_list),len(income_full_id_list)

(709, 893)

In [24]:
##验证
df1=job_info[inc_code_list].replace([-5,-4,-3,-2,-1,0], np.nan)
df1.dropna(axis=0,how='any').shape[0]

893

In [27]:
## occu + 1/2/3
# event number >=1
len([i for i in have_1st_job_list if i['event_n']>0 and i['person_id'] in occu_full_id_list]) #698
# event number >=2
len([i for i in have_1st_job_list if i['event_n']>1 and i['person_id'] in occu_full_id_list]) #677
# event number >=3
len([i for i in have_1st_job_list if i['event_n']>2 and i['person_id'] in occu_full_id_list ]) # 621

698

In [30]:
## income + 1/2/3
# event number >=1
len([i for i in have_1st_job_list if i['event_n']>0 and i['person_id'] in income_full_id_list]) #881
# event number >=2
len([i for i in have_1st_job_list if i['event_n']>1 and i['person_id'] in income_full_id_list]) #859
# event number >=3
len([i for i in have_1st_job_list if i['event_n']>2 and i['person_id'] in income_full_id_list ]) # 806

881